## Биомедицинские нанотехнологии и компьютерное зрение (Computer Vision) 
<br>

### 3. Трекинг объектов на изображении

In [1]:
# Video analysis (video module)
# https://docs.opencv.org/3.4/da/dd0/tutorial_table_of_content_video.html

<div align='justify'>В данном модуле рассмотрим ещё одно важное понятие в области компьютерного зрения, которое называется трекингом или отслеживанием (Tracking). Отслеживание - это задача определения положения объекта на последовательности изображений. Чаще всего данная задача применима именно к потоковому видео. В частности, отслеживание может быть выполнено в случае детекции одного или нескольких объектов одновременно. В этом модуле мы рассмотрим оба метода отслеживания объектов на изображении.</div>

<img width='35%' src='img/cell_move_short.gif'>

<div align='justify'>Методы отслеживания нашли широкое применение в таких приложениях, как распознавание действий, автомобили с автопилотом, охрана и видеонаблюдение, приложения дополненной реальности (AR) и системы детекции движения. Например, в приложениях дополненной реальности, когда необходимо нарисовать трёхмерный объект на плоскости. Достаточно распространённой задачей является мониторинг дорожного движения, отслеживание транспортных средств и учет номерных знаков, что позволяет управлять дорожным движением и контролировать безопасность.<br>
В биологии и медицине данный подход можно использовать для анализа движений микроорганизмов и детекции движения структурных элементов различных жидкостей.
</div>

<div align='justify'><b>Основные проблемы трекинга.</b><br>

• Скрытие объекта. Целевой объект может быть скрыт за другими объектами в последовательности изображений. Для отслеживание требуется его  выделение на фоне остальных объектов.

• Быстрое перемещение. Перемещение объекта может приводить к размытию границ между объектом и фоном. В этом случае требуется использование дорогих камер, позволяющих снимать видео в высоком разрешении. 

• Изменение формы. Если объект способен изменять форму, то подобная деформация приводит к невозможности обнаружения объекта, а также к проблеме отслеживания. 

• Ложные срабатывания. Га изображении с несколькими близкими по форме объектами трудно выделить тот, который нас интересует. Трекер может потерять текущий объект и начать отслеживать ложный объект.
</div><br>
<div align='justify'>
Далее будет рассмотрена программная реализация методов трекинга объектов в библиотеке OpenCV. Мы рассмотрим трекинг на основе вычитания фона (Background Subtraction) и на основе среднего сдвига (Meanshift).
</div>

<div align='justify'>Перед рассмотрением методов детекции объектов создадим тестовое видео с движущимся объектом. Будем использовать следующую программу:<div>

In [2]:
# Программа для создания тестового изображения

import cv2 as cv
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Создание изображения
# fig = plt.figure(figsize=(5, 5))                      
# ax = fig.add_subplot(1, 1, 1)

# Будем использовать окружность с диаметром 2.5 у.е.
# Траектория движения описывается тригонометрической функцией синуса с частотой f = 1 Гц
# Функция для изменения координаты объекта на изображении
def ani_func(i):
    ax.clear()
    x = i*0.01
    y = 2.5*math.sin(2*3.14*x)
    circle = plt.Circle((x, y), 0.25, color='g')
    plt.gca().add_patch(circle)
    ax.set_xlim([0, 10])
    ax.set_ylim([-5, 5])
    ax.axis('off')

# Анимирование изображения и сохранение в файл *.mp4
# ani = animation.FuncAnimation(fig, ani_func, frames=1000, interval=10, repeat=False)
# ani.save('animate_circle_sin.mp4')

### 3.1 Отслеживание траектории движения на основе вычитания фона (Background Subtraction)

In [3]:
# Background Subtraction Methods 
# https://docs.opencv.org/3.4/d1/dc5/tutorial_background_subtraction.html

<div align='justify'>Вычитание фона (Background Subtraction) - это метод для создания маски переднего плана (точнее двоичного изображения, содержащего пиксели движущимся объектам в сцене), основанный на использовании статичных камер.
Метод вычитания фона вычисляет маску переднего плана, выполняя вычитание между текущим кадром и фоновым изображением (background model). Результатом обработки является выделение объекта на изображении. После бинаризации интересуемый объект будет белым. Ниже показан результат работы метода вычитания.</div>

<img src='img/bst_scheme.png'>

<div align='justify'>На первом шаге вычисляется модель (фон) для объекта на изображении. На втором шаге данная модель обновляется, чтобы адаптироваться к возможным изменениям на изображении. Далее рассмотрим пример программы для трекинга на основе вычитания фона (Background Subtraction).</div>

In [4]:
# Программа для трекинга на основе вычитания фона (Background Subtraction)

# Создание объекта
fgbg = cv.createBackgroundSubtractorMOG2()

# Захват кадров из видео
capture = cv.VideoCapture('data/lab_3/animate_circle_sin.mp4')

while True:

    # Чтение кадров
    ret, frame = capture.read();
    if frame is None:
        break
        
    # Применение маски для вычитания фона
    fgmask = fgbg.apply(frame)
    cv.imshow('Original image', frame)
    cv.imshow('MOG image', fgmask)
   
    # Выход из потока при нажатии `q`
    if cv.waitKey(1) == ord('q'):
        break

capture.release()
cv.destroyAllWindows()

### 3.2 Отслеживание траектории движения на основе поиска и анализа контуров

In [5]:
# Программа для трекинга на основе анализа контуров

# Создание объекта
fgbg = cv.createBackgroundSubtractorMOG2()

# Захват кадров из видео
capture = cv.VideoCapture('data/lab_3/animate_circle_sin.mp4')

data = []

while True:

    # Чтение кадров
    ret, frame = capture.read();
    if frame is None:
        break

    # Преобразование кадра BGR -> RGB -> GRAY
    img_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    imgray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)

    # Поиск контуров на изображении
    ret, thresh = cv.threshold(imgray, 155, 255, 0)
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cnt = contours[1]
    
    # Отрисовка окружности    
    (x,y), radius = cv.minEnclosingCircle(cnt)
    center = (int(x), int(y))
    radius = int(radius) + 5
    cv.circle(frame, center, radius, (0, 255, 0), 2)
    
    # Отрисовка прямоугольника
    x, y, w, h = cv.boundingRect(cnt)
    cv.rectangle(frame, (x,y), (x+h,y+w), (0, 255, 0), 1)
    
    # Вывод на экран координат центра объекта
    cv.putText(frame, "circle", (10, 10),
                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    
    cv.putText(frame, f'x, y = {round(x, 2), round(y, 2)}', (20, 40),
                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    
    # Отрисовка траектории движения
    data.append([x, y])
    image = cv.polylines(frame, [np.array(data) ], False, (255, 0, 0), 2)
    
    cv.imshow(f'Original video', img_rgb)
    cv.imshow(f'Tracking video', frame)
    
    # Выход из потока при нажатии `q`
    if cv.waitKey(1) == ord('q'):
        break

capture.release()
cv.destroyAllWindows()

### 3.3 Отслеживание траектории движения на основе анализа среднего значения (Meanshift)

In [6]:
# Meanshift and Camshift
# https://docs.opencv.org/3.4/d7/d00/tutorial_meanshift.html

<div align='justify'>Сдвиг среднего значения (Meanshift) - метод отслеживания траектории движения объекта, который основан на анализе плотности распределения пикселей на изображении. На изображении выделяется небольшое окно в виде круга. Задача программы переместить круг в область, которая имеет максимальную плотность.</div>

<img width='60%' src='img/meanshift.png'>

<div align='justify'>Исходное положение окна представлено в виде синего круга "C1", а его изначальный центр обозначен как синий прямоугольник "C1_о". Если найти центроид всех точек в пределах этого окна, то мы получим точку "C1_r", обозначенную маленьким синим кругом. Безусловно, эти две точки не будут совпадать. Поэтому нам следует переместить окно таким образом, чтобы центр нового окна совпадал с предыдущим центроидом. Затем мы вновь находим новый центроид и повторяем процесс до тех пор, пока центр окна и его центроид не окажутся в одной и той же точке в пределах погрешности. В итоге мы получаем окно с наибольшим распределением пикселей, обозначенное зелёным кругом с пометкой "C2", которое имеет наибольшую плотность пикселей, как это видно на изображении.</div>

<div align='justify'>Далее рассмотрим пример программы для трекинга на основе сдвига среднего значения (Meanshift).</div>

In [7]:
# Программа для трекинга на основе анализа среднего значения

# Захват кадров из видео
capture = cv.VideoCapture('data/lab_3/animate_circle_sin.mp4')
# capture = cv2.VideoCapture('data/lab_3/cell_move_short.mp4')


# Чтение кадров для отслеживаемого кадра по ROI
# ROI - region of interest (интересуемая область)
ret, frame = capture.read()

# Создание ROI
x,y,w,h = cv.selectROI(frame)
track_window = (x, y, w, h)
roi = frame[y:y+h, x:x+w]
hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)

# Применение ROI
mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)
term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

while True:
    # Чтение кадров
    ret, frame = capture.read()
    if ret == True:
        
        # Перевод из схемы BGR в HSV
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        # Смещение трекера на изображении
        ret, track_window = cv.meanShift(dst, track_window, term_crit)
        x,y,w,h = track_window
        img = cv.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv.imshow('img', img)  
        
        # Выход из потока при нажатии `q`
        if cv.waitKey(1) == ord('q'):
            break
    else:
        break

capture.release()
cv.destroyAllWindows()

<div align='justify'>Рекомендуется самостоятельно разобрать метод на основе анализа оптического потока (Optical Flow) и на основе корреляционной фильтрации (Kernelized Correlation Filter). Для пакетного использования трекеров можно воспользоваться Tracking API</div>

### Задание для самостоятельной работы.

1. Объясните, что такое трекинг и где он может быть использован. Перечислите основные проблемы, которые связаны с трекингом. Какие основные методы для определения траектории движения вы можете назвать.

2. Создайте с помощью функции ani_func() функцию, которая позволит смоделировать траекторию движение квадрата. Траектория движения описывается тригонометрической функцией косинуса с частотой f = 0.5 Гц, амплитудой А = 4 от.ед. Частота дискритизации  кадров анимации составляет 100 Гц.

3. Проанализируйте с помощью библиотеки OpenCV и метода вычитания фона (Background Subtraction) тестовые видео. Предлагается два видео, на которых необходимо отследить траекторию движения объекта(ов). Используйте видео animate_1_circle.mp4 и animate_2_circle.mp4.

4. Проанализируйте с помощью библиотеки OpenCV и метода контурного анализа тестовые видео. Предлагается три видео, на которых необходимо отследить траекторию движения объекта. Далее нужно сохранить траектории движения для каждого видео. Потом проанализировать полученные ряды, используя преобразование Фурье. Определить основные частоты в спектре. Частота дискритизации кадров анимации составляет 100 Гц. Используйте видео circle_1_sample.mp4, circle_2_sample.mp4 и circle_3_sample.mp4.

5. Проанализируйте с помощью библиотеки OpenCV и метода на основе анализа среднего значения (Meanshift) тестовое видео. Выберите для отслеживания траектории любой объект. Сохраните траекторию движения в отдельный файл. Используйте видео cell_move_short.mp4. 